In [298]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [299]:
%autoreload 2

In [300]:
from lib import sim_env as sim
from lib import loads
from lib.utils import (DTParser, current_time_tz, 
                       dt_to_timezone, adjust_db_dates_local_tz)
from lib import agents
import datetime

In [301]:
dt_parser = DTParser()

In [302]:
dt = datetime.datetime(2023, 11, 21)
dt

datetime.datetime(2023, 11, 21, 0, 0)

In [303]:
dt = current_time_tz(cut_microseconds=True)
dt

datetime.datetime(2024, 3, 15, 15, 23, 44, tzinfo=datetime.timezone.utc)

In [304]:
env = sim.SimulationEnvironment(name='base', time_unit='seconds', starting_datetime=dt)
job_generator = loads.RandomJobGenerator(seed=2)
infstruct_mgr = sim.InfrastructureManager(env=env)
dispatcher = sim.Dispatcher(env=env, priority_rule='FIFO')

INFO:sim_env.env:Successfully registered Infrastructure Manager in Env >>base<<
INFO:sim_env.env:Successfully registered Dispatcher in Env >>base<<


In [305]:
# source
area_source = sim.ProductionArea(env=env, custom_identifier=1000)
group_source = sim.StationGroup(env=env, custom_identifier=1000)
area_source.add_subsystem(group_source)
td = dt_parser.timedelta_from_val(val=2., time_unit='hours')
source = sim.Source(env=env, custom_identifier='source', proc_time=td, 
                random_generation=True, job_generator=job_generator, num_gen_jobs=5)
group_source.add_subsystem(source)

# sink
area_sink = sim.ProductionArea(env=env, custom_identifier=2000)
group_sink = sim.StationGroup(env=env, custom_identifier=2000)
area_sink.add_subsystem(group_sink)
sink = sim.Sink(env=env, custom_identifier='sink')
group_sink.add_subsystem(sink)

# processing stations
# prod area 1
area_prod = sim.ProductionArea(env=env, custom_identifier=1)
group_prod = sim.StationGroup(env=env, custom_identifier=1)
area_prod.add_subsystem(group_prod)
group_prod2 = sim.StationGroup(env=env, custom_identifier=2)
area_prod.add_subsystem(group_prod2)
# prod area 2
#area_prod2 = ProductionArea(env=env, custom_identifier=2)
#group_prod3 = StationGroup(env=env, custom_identifier=3)
#area_prod2.add_subsystem(group_prod3)
#area_prod.add_subsystem(group_prod3)
## machines
for machine in range(3):
    buffer = sim.Buffer(capacity=20, env=env, custom_identifier=(10+machine))
    if machine == 5:
        MachInst = sim.Machine(env=env, custom_identifier=machine, buffers=[buffer], setup_time=5.)
    else:
        MachInst = sim.Machine(env=env, custom_identifier=machine, buffers=[buffer])
        
    if machine == 0:
        testMachInst = MachInst
    
    if machine < 2:
        group_prod.add_subsystem(buffer)
        group_prod.add_subsystem(MachInst)
    elif machine >= 2:
        group_prod2.add_subsystem(buffer)
        group_prod2.add_subsystem(MachInst)
    else:
        pass
        #group_prod3.add_subsystem(buffer)
        #group_prod3.add_subsystem(MachInst)
    

add_machine_to_bottleneck: bool = False
if add_machine_to_bottleneck:
    buffer = sim.Buffer(capacity=20, env=env, custom_identifier=(10+machine+1))
    MachInst = sim.Machine(env=env, custom_identifier=machine+1, buffers=[buffer])
    group_prod3.add_subsystem(buffer)
    group_prod3.add_subsystem(MachInst)

INFO:sim_env.infstrct:Successfully registered object with SystemID 0 and name ProductionArea_env_0
INFO:sim_env.infstrct:Successfully registered object with SystemID 0 and name StationGroup_env_0
INFO:sim_env.infstrct:Successfully added System (type: StationGroup, custom_id: 1000, name: StationGroup_env_0) to System (type: ProductionArea, custom_id: 1000, name: ProductionArea_env_0).
INFO:sim_env.infstrct:Successfully registered object with SystemID 0 and name Source_env_0
INFO:sim_env.infstrct:Successfully added System (type: Resource, custom_id: source, name: Source_env_0) to System (type: StationGroup, custom_id: 1000, name: StationGroup_env_0).
INFO:sim_env.infstrct:Successfully registered object with SystemID 1 and name ProductionArea_env_1
INFO:sim_env.infstrct:Successfully registered object with SystemID 1 and name StationGroup_env_1
INFO:sim_env.infstrct:Successfully added System (type: StationGroup, custom_id: 2000, name: StationGroup_env_1) to System (type: ProductionArea, cu

In [306]:
alloc_agent = agents.AllocationAgent(assoc_system=area_prod)

INFO:sim_env.env:Successfully registered Allocation Agent in System (type: ProductionArea, custom_id: 1, name: ProductionArea_env_2)


In [307]:
alloc_agent._assoc_infstrct_objs

(System (type: Resource, custom_id: 0, name: Machine_env_3),
 System (type: Resource, custom_id: 1, name: Machine_env_5),
 System (type: Resource, custom_id: 2, name: Machine_env_7))

In [308]:
# station group IDs (system, custom) for processing stations
MachInst.supersystems_custom_ids

{2}

In [309]:
infstruct_mgr.prod_area_db

,custom_id,name,prod_area,containing_proc_stations
prod_area_id,,,,
0,1000,ProductionArea_env_0,{0: {0: {}}},False
1,2000,ProductionArea_env_1,{1: {1: {}}},False
2,1,ProductionArea_env_2,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",True


In [310]:
infstruct_mgr.station_group_db

,custom_id,name,station_group,prod_area_id,containing_proc_stations
station_group_id,,,,,
0,1000,StationGroup_env_0,{0: {}},0,False
1,2000,StationGroup_env_1,{1: {}},1,False
2,1,StationGroup_env_2,"{2: {}, 3: {}, 4: {}, 5: {}}",2,True
3,2,StationGroup_env_3,"{6: {}, 7: {}}",2,True


In [311]:
infstruct_mgr.res_db

,custom_id,resource,name,res_type,state,station_group_id
res_id,,,,,,
0,source,{},Source_env_0,Source,INIT,0
1,sink,{},Sink_env_1,Sink,INIT,1
2,10,{},Buffer_env_2,Buffer,INIT,2
3,0,{},Machine_env_3,Machine,INIT,2
4,11,{},Buffer_env_4,Buffer,INIT,2
5,1,{},Machine_env_5,Machine,INIT,2
6,12,{},Buffer_env_6,Buffer,INIT,3
7,2,{},Machine_env_7,Machine,INIT,3


In [312]:
env.check_integrity()

INFO:sim_env.env:Integrity check for Environment base successful.


In [313]:
print('Prio Rules:', dispatcher.possible_prio_rules())
print('Alloc Rules:', dispatcher.possible_alloc_rules())

Prio Rules: {'LPT', 'LIFO', 'FIFO', 'LST', 'SST', 'PRIO', 'SPT'}
Alloc Rules: {'RANDOM', 'WIP_LOAD_TIME', 'UTILISATION', 'WIP_LOAD_JOBS', 'AGENT'}


In [314]:
dispatcher.curr_prio_rule = 'LPT'
dispatcher.curr_alloc_rule = 'WIP_LOAD_TIME'

INFO:sim_env.dispatcher:Changed priority rule to LPT
INFO:sim_env.dispatcher:Changed allocation rule to WIP_LOAD_TIME


In [315]:
env.run()

INFO:sim_env.dispatcher:Successfully registered job with JobID 0 and name J_gen_0
INFO:sim_env.dispatcher:Successfully registered operation with OpID 0 and name O_gen_0
--------------- DEBUG: call before hold(0) at 0.0
--------------- DEBUG: call after hold(0) at 0.0
INFO:sim_env.dispatcher:[DISPATCHER: Dispatcher(env: base)] REQUEST TO DISPATCHER FOR ALLOCATION
DEBUG:sim_env.dispatcher:[DISPATCHER: Dispatcher(env: base)] Next operation Operation(ProcTime: 6:00:00, ExecutionSystemID: 1, SGI: 1)
DEBUG:sim_env.dispatcher:[DISPATCHER: Dispatcher(env: base)] Available stations at 0.0 are [System (type: Resource, custom_id: 0, name: Machine_env_3), System (type: Resource, custom_id: 1, name: Machine_env_5)]
DEBUG:sim_env.dispatcher:[DISPATCHER: Dispatcher(env: base)] WIP LOAD TIME of target_station=System (type: Resource, custom_id: 0, name: Machine_env_3) is 0:00:00
DEBUG:sim_env.dispatcher:[DISPATCHER: Dispatcher(env: base)] Next operation is Operation(ProcTime: 6:00:00, ExecutionSystemID

In [316]:
env.finalise()

INFO:sim_env.infstrct:Successful finalisation of the state information for all resource objects.


In [317]:
dispatcher.op_db.head()

,job_id,job_name,custom_id,op,name,prio,execution_system,execution_system_custom_id,execution_system_name,execution_system_type,...,creation_date,release_date,planned_starting_date,actual_starting_date,starting_date_deviation,planned_ending_date,actual_ending_date,ending_date_deviation,lead_time,state
op_id,,,,,,,,,,,,,,,,,,,,,
0,0,J_gen_0,None,"Operation(ProcTime: 6:00:00, ExecutionSystemID...",O_gen_0,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 15:23:44+00:00,2024-03-15 15:23:44+00:00,None,2024-03-15 15:23:44+00:00,None,None,2024-03-15 22:23:44+00:00,None,7:00:00,FINISH
1,1,J_gen_1,None,"Operation(ProcTime: 9:00:00, ExecutionSystemID...",O_gen_1,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 17:23:44+00:00,2024-03-15 17:23:44+00:00,None,2024-03-15 17:23:44+00:00,None,None,2024-03-16 09:23:44+00:00,None,16:00:00,FINISH
2,2,J_gen_2,None,"Operation(ProcTime: 7:00:00, ExecutionSystemID...",O_gen_2,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 19:23:44+00:00,2024-03-15 19:23:44+00:00,None,2024-03-15 19:23:44+00:00,None,None,2024-03-16 10:23:44+00:00,None,15:00:00,FINISH
3,3,J_gen_3,None,"Operation(ProcTime: 5:00:00, ExecutionSystemID...",O_gen_3,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 21:23:44+00:00,2024-03-15 21:23:44+00:00,None,2024-03-15 22:23:44+00:00,None,None,2024-03-16 05:23:44+00:00,None,8:00:00,FINISH
4,4,J_gen_4,None,"Operation(ProcTime: 9:00:00, ExecutionSystemID...",O_gen_4,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 23:23:44+00:00,2024-03-15 23:23:44+00:00,None,2024-03-16 05:23:44+00:00,None,None,2024-03-16 17:23:44+00:00,None,18:00:00,FINISH


In [318]:
dispatcher.job_db

,custom_id,job,name,job_type,prio,total_proc_time,creation_date,release_date,planned_starting_date,actual_starting_date,starting_date_deviation,planned_ending_date,actual_ending_date,ending_date_deviation,lead_time,state
job_id,,,,,,,,,,,,,,,,
0,None,Job (J_gen_0),J_gen_0,Job,1,0 days 06:00:00,2024-03-15 15:23:44+00:00,2024-03-15 15:23:44+00:00,2023-11-20 06:00:00+00:00,2024-03-15 15:23:44+00:00,"116 days, 9:23:44",2023-12-01 10:00:00+00:00,2024-03-15 22:23:44+00:00,"105 days, 12:23:44",7:00:00,FINISH
1,None,Job (J_gen_1),J_gen_1,Job,8,0 days 09:00:00,2024-03-15 17:23:44+00:00,2024-03-15 17:23:44+00:00,2023-11-20 06:00:00+00:00,2024-03-15 17:23:44+00:00,"116 days, 11:23:44",2023-12-01 10:00:00+00:00,2024-03-16 09:23:44+00:00,"105 days, 23:23:44",16:00:00,FINISH
2,None,Job (J_gen_2),J_gen_2,Job,5,0 days 07:00:00,2024-03-15 19:23:44+00:00,2024-03-15 19:23:44+00:00,2023-11-20 06:00:00+00:00,2024-03-15 19:23:44+00:00,"116 days, 13:23:44",2023-12-01 10:00:00+00:00,2024-03-16 10:23:44+00:00,"106 days, 0:23:44",15:00:00,FINISH
3,None,Job (J_gen_3),J_gen_3,Job,9,0 days 05:00:00,2024-03-15 21:23:44+00:00,2024-03-15 21:23:44+00:00,2023-11-20 06:00:00+00:00,2024-03-15 22:23:44+00:00,"116 days, 16:23:44",2023-12-01 10:00:00+00:00,2024-03-16 05:23:44+00:00,"105 days, 19:23:44",8:00:00,FINISH
4,None,Job (J_gen_4),J_gen_4,Job,12,0 days 09:00:00,2024-03-15 23:23:44+00:00,2024-03-15 23:23:44+00:00,2023-11-20 06:00:00+00:00,2024-03-16 05:23:44+00:00,"116 days, 23:23:44",2023-12-01 10:00:00+00:00,2024-03-16 17:23:44+00:00,"106 days, 7:23:44",18:00:00,FINISH


In [321]:
fig = dispatcher.draw_gantt_chart(dates_to_local_tz=True)

**Datetime conversion**

In [296]:
dispatcher.op_db_date_adjusted

,job_id,job_name,custom_id,op,name,prio,execution_system,execution_system_custom_id,execution_system_name,execution_system_type,...,creation_date,release_date,planned_starting_date,actual_starting_date,starting_date_deviation,planned_ending_date,actual_ending_date,ending_date_deviation,lead_time,state
op_id,,,,,,,,,,,,,,,,,,,,,
0,0,J_gen_0,None,"Operation(ProcTime: 6:00:00, ExecutionSystemID...",O_gen_0,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 16:21:45+01:00,2024-03-15 16:21:45+01:00,None,2024-03-15 16:21:45+01:00,None,None,2024-03-15 23:21:45+01:00,None,7:00:00,FINISH
1,1,J_gen_1,None,"Operation(ProcTime: 9:00:00, ExecutionSystemID...",O_gen_1,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 18:21:45+01:00,2024-03-15 18:21:45+01:00,None,2024-03-15 18:21:45+01:00,None,None,2024-03-16 10:21:45+01:00,None,16:00:00,FINISH
2,2,J_gen_2,None,"Operation(ProcTime: 7:00:00, ExecutionSystemID...",O_gen_2,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 20:21:45+01:00,2024-03-15 20:21:45+01:00,None,2024-03-15 20:21:45+01:00,None,None,2024-03-16 11:21:45+01:00,None,15:00:00,FINISH
3,3,J_gen_3,None,"Operation(ProcTime: 5:00:00, ExecutionSystemID...",O_gen_3,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 22:21:45+01:00,2024-03-15 22:21:45+01:00,None,2024-03-15 23:21:45+01:00,None,None,2024-03-16 06:21:45+01:00,None,8:00:00,FINISH
4,4,J_gen_4,None,"Operation(ProcTime: 9:00:00, ExecutionSystemID...",O_gen_4,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-16 00:21:45+01:00,2024-03-16 00:21:45+01:00,None,2024-03-16 06:21:45+01:00,None,None,2024-03-16 18:21:45+01:00,None,18:00:00,FINISH


In [297]:
dispatcher.job_db_date_adjusted

,custom_id,job,name,job_type,prio,total_proc_time,creation_date,release_date,planned_starting_date,actual_starting_date,starting_date_deviation,planned_ending_date,actual_ending_date,ending_date_deviation,lead_time,state
job_id,,,,,,,,,,,,,,,,
0,None,Job (J_gen_0),J_gen_0,Job,1,0 days 06:00:00,2024-03-15 16:21:45+01:00,2024-03-15 16:21:45+01:00,2023-11-20 07:00:00+01:00,2024-03-15 16:21:45+01:00,"116 days, 9:21:45",2023-12-01 11:00:00+01:00,2024-03-15 23:21:45+01:00,"105 days, 12:21:45",7:00:00,FINISH
1,None,Job (J_gen_1),J_gen_1,Job,8,0 days 09:00:00,2024-03-15 18:21:45+01:00,2024-03-15 18:21:45+01:00,2023-11-20 07:00:00+01:00,2024-03-15 18:21:45+01:00,"116 days, 11:21:45",2023-12-01 11:00:00+01:00,2024-03-16 10:21:45+01:00,"105 days, 23:21:45",16:00:00,FINISH
2,None,Job (J_gen_2),J_gen_2,Job,5,0 days 07:00:00,2024-03-15 20:21:45+01:00,2024-03-15 20:21:45+01:00,2023-11-20 07:00:00+01:00,2024-03-15 20:21:45+01:00,"116 days, 13:21:45",2023-12-01 11:00:00+01:00,2024-03-16 11:21:45+01:00,"106 days, 0:21:45",15:00:00,FINISH
3,None,Job (J_gen_3),J_gen_3,Job,9,0 days 05:00:00,2024-03-15 22:21:45+01:00,2024-03-15 22:21:45+01:00,2023-11-20 07:00:00+01:00,2024-03-15 23:21:45+01:00,"116 days, 16:21:45",2023-12-01 11:00:00+01:00,2024-03-16 06:21:45+01:00,"105 days, 19:21:45",8:00:00,FINISH
4,None,Job (J_gen_4),J_gen_4,Job,12,0 days 09:00:00,2024-03-16 00:21:45+01:00,2024-03-16 00:21:45+01:00,2023-11-20 07:00:00+01:00,2024-03-16 06:21:45+01:00,"116 days, 23:21:45",2023-12-01 11:00:00+01:00,2024-03-16 18:21:45+01:00,"106 days, 7:21:45",18:00:00,FINISH


In [99]:
df

,job_id,job_name,custom_id,op,name,prio,execution_system,execution_system_custom_id,execution_system_name,execution_system_type,...,creation_date,release_date,planned_starting_date,actual_starting_date,starting_date_deviation,planned_ending_date,actual_ending_date,ending_date_deviation,lead_time,state
op_id,,,,,,,,,,,,,,,,,,,,,
0,0,J_gen_0,None,"Operation(ProcTime: 6:00:00, ExecutionSystemID...",O_gen_0,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 15:06:32+00:00,2024-03-15 15:06:32+00:00,None,2024-03-15 15:06:32+00:00,None,None,2024-03-15 22:06:32+00:00,None,7:00:00,FINISH
1,1,J_gen_1,None,"Operation(ProcTime: 9:00:00, ExecutionSystemID...",O_gen_1,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 17:06:32+00:00,2024-03-15 17:06:32+00:00,None,2024-03-15 17:06:32+00:00,None,None,2024-03-16 09:06:32+00:00,None,16:00:00,FINISH
2,2,J_gen_2,None,"Operation(ProcTime: 7:00:00, ExecutionSystemID...",O_gen_2,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 19:06:32+00:00,2024-03-15 19:06:32+00:00,None,2024-03-15 19:06:32+00:00,None,None,2024-03-16 10:06:32+00:00,None,15:00:00,FINISH
3,3,J_gen_3,None,"Operation(ProcTime: 5:00:00, ExecutionSystemID...",O_gen_3,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 21:06:32+00:00,2024-03-15 21:06:32+00:00,None,2024-03-15 22:06:32+00:00,None,None,2024-03-16 05:06:32+00:00,None,8:00:00,FINISH
4,4,J_gen_4,None,"Operation(ProcTime: 9:00:00, ExecutionSystemID...",O_gen_4,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 23:06:32+00:00,2024-03-15 23:06:32+00:00,None,2024-03-16 05:06:32+00:00,None,None,2024-03-16 17:06:32+00:00,None,18:00:00,FINISH


In [100]:
db = adjust_db_dates_local_tz(db=df)

In [101]:
db

,job_id,job_name,custom_id,op,name,prio,execution_system,execution_system_custom_id,execution_system_name,execution_system_type,...,creation_date,release_date,planned_starting_date,actual_starting_date,starting_date_deviation,planned_ending_date,actual_ending_date,ending_date_deviation,lead_time,state
op_id,,,,,,,,,,,,,,,,,,,,,
0,0,J_gen_0,None,"Operation(ProcTime: 6:00:00, ExecutionSystemID...",O_gen_0,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 16:06:32+01:00,2024-03-15 16:06:32+01:00,None,2024-03-15 16:06:32+01:00,None,None,2024-03-15 23:06:32+01:00,None,7:00:00,FINISH
1,1,J_gen_1,None,"Operation(ProcTime: 9:00:00, ExecutionSystemID...",O_gen_1,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 18:06:32+01:00,2024-03-15 18:06:32+01:00,None,2024-03-15 18:06:32+01:00,None,None,2024-03-16 10:06:32+01:00,None,16:00:00,FINISH
2,2,J_gen_2,None,"Operation(ProcTime: 7:00:00, ExecutionSystemID...",O_gen_2,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 20:06:32+01:00,2024-03-15 20:06:32+01:00,None,2024-03-15 20:06:32+01:00,None,None,2024-03-16 11:06:32+01:00,None,15:00:00,FINISH
3,3,J_gen_3,None,"Operation(ProcTime: 5:00:00, ExecutionSystemID...",O_gen_3,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 22:06:32+01:00,2024-03-15 22:06:32+01:00,None,2024-03-15 23:06:32+01:00,None,None,2024-03-16 06:06:32+01:00,None,8:00:00,FINISH
4,4,J_gen_4,None,"Operation(ProcTime: 9:00:00, ExecutionSystemID...",O_gen_4,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-16 00:06:32+01:00,2024-03-16 00:06:32+01:00,None,2024-03-16 06:06:32+01:00,None,None,2024-03-16 18:06:32+01:00,None,18:00:00,FINISH


In [57]:
date_cols = get_date_cols(df=df)

In [58]:
df[date_cols]

,creation_date,release_date,planned_starting_date,actual_starting_date,starting_date_deviation,planned_ending_date,actual_ending_date,ending_date_deviation
op_id,,,,,,,,
0,2024-03-15 14:52:35+00:00,2024-03-15 14:52:35+00:00,None,2024-03-15 14:52:35+00:00,None,None,2024-03-15 21:52:35+00:00,None
1,2024-03-15 16:52:35+00:00,2024-03-15 16:52:35+00:00,None,2024-03-15 16:52:35+00:00,None,None,2024-03-16 08:52:35+00:00,None
2,2024-03-15 18:52:35+00:00,2024-03-15 18:52:35+00:00,None,2024-03-15 18:52:35+00:00,None,None,2024-03-16 09:52:35+00:00,None
3,2024-03-15 20:52:35+00:00,2024-03-15 20:52:35+00:00,None,2024-03-15 21:52:35+00:00,None,None,2024-03-16 04:52:35+00:00,None
4,2024-03-15 22:52:35+00:00,2024-03-15 22:52:35+00:00,None,2024-03-16 04:52:35+00:00,None,None,2024-03-16 16:52:35+00:00,None


In [65]:
columns = [
    'planned_starting_date',
    'actual_starting_date',
    'planned_ending_date',
    'actual_ending_date',
]

In [66]:
df_filt = df[columns]

In [76]:
df_filt = df_filt.applymap(dt_to_timezone, na_action='ignore')

In [77]:
df_filt

,planned_starting_date,actual_starting_date,planned_ending_date,actual_ending_date
op_id,,,,
0,None,2024-03-15 15:34:27+01:00,None,2024-03-15 22:34:27+01:00
1,None,2024-03-15 17:34:27+01:00,None,2024-03-16 09:34:27+01:00
2,None,2024-03-15 19:34:27+01:00,None,2024-03-16 10:34:27+01:00
3,None,2024-03-15 22:34:27+01:00,None,2024-03-16 05:34:27+01:00
4,None,2024-03-16 05:34:27+01:00,None,2024-03-16 17:34:27+01:00


In [78]:
df[columns] = df_filt

In [79]:
df

,job_id,job_name,custom_id,op,name,prio,execution_system,execution_system_custom_id,execution_system_name,execution_system_type,...,creation_date,release_date,planned_starting_date,actual_starting_date,starting_date_deviation,planned_ending_date,actual_ending_date,ending_date_deviation,lead_time,state
op_id,,,,,,,,,,,,,,,,,,,,,
0,0,J_gen_0,None,"Operation(ProcTime: 6:00:00, ExecutionSystemID...",O_gen_0,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 14:34:27+00:00,2024-03-15 14:34:27+00:00,None,2024-03-15 15:34:27+01:00,None,None,2024-03-15 22:34:27+01:00,None,7:00:00,FINISH
1,1,J_gen_1,None,"Operation(ProcTime: 9:00:00, ExecutionSystemID...",O_gen_1,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 16:34:27+00:00,2024-03-15 16:34:27+00:00,None,2024-03-15 17:34:27+01:00,None,None,2024-03-16 09:34:27+01:00,None,16:00:00,FINISH
2,2,J_gen_2,None,"Operation(ProcTime: 7:00:00, ExecutionSystemID...",O_gen_2,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 18:34:27+00:00,2024-03-15 18:34:27+00:00,None,2024-03-15 19:34:27+01:00,None,None,2024-03-16 10:34:27+01:00,None,15:00:00,FINISH
3,3,J_gen_3,None,"Operation(ProcTime: 5:00:00, ExecutionSystemID...",O_gen_3,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 20:34:27+00:00,2024-03-15 20:34:27+00:00,None,2024-03-15 22:34:27+01:00,None,None,2024-03-16 05:34:27+01:00,None,8:00:00,FINISH
4,4,J_gen_4,None,"Operation(ProcTime: 9:00:00, ExecutionSystemID...",O_gen_4,None,"{2: {2: {}, 3: {}, 4: {}, 5: {}}, 3: {6: {}, 7...",1,<bound method System.name of System (type: Pro...,ProductionArea,...,2024-03-15 22:34:27+00:00,2024-03-15 22:34:27+00:00,None,2024-03-16 05:34:27+01:00,None,None,2024-03-16 17:34:27+01:00,None,18:00:00,FINISH


In [81]:
lst = list()
for col in df.columns:
    if 'date' in col:
        lst.append(col)

In [82]:
lst

['creation_date',
 'release_date',
 'planned_starting_date',
 'actual_starting_date',
 'starting_date_deviation',
 'planned_ending_date',
 'actual_ending_date',
 'ending_date_deviation']

In [67]:
filt_na = df[columns].isna().any(axis=0)

In [68]:
df_filt2 = df_filt.loc[:,~filt_na]

In [70]:
df_filt2

,actual_starting_date,actual_ending_date
op_id,,
0,2024-03-15 14:34:27+00:00,2024-03-15 21:34:27+00:00
1,2024-03-15 16:34:27+00:00,2024-03-16 08:34:27+00:00
2,2024-03-15 18:34:27+00:00,2024-03-16 09:34:27+00:00
3,2024-03-15 21:34:27+00:00,2024-03-16 04:34:27+00:00
4,2024-03-16 04:34:27+00:00,2024-03-16 16:34:27+00:00


In [74]:
df_filt2.applymap(dt_to_timezone)

,actual_starting_date,actual_ending_date
op_id,,
0,2024-03-15 15:34:27+01:00,2024-03-15 22:34:27+01:00
1,2024-03-15 17:34:27+01:00,2024-03-16 09:34:27+01:00
2,2024-03-15 19:34:27+01:00,2024-03-16 10:34:27+01:00
3,2024-03-15 22:34:27+01:00,2024-03-16 05:34:27+01:00
4,2024-03-16 05:34:27+01:00,2024-03-16 17:34:27+01:00
